In [178]:
import pandas as pd
import os 
import numpy as np 
import librosa
from tqdm import tqdm

In [179]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 44CF-1D40

 Directory of c:\Users\admin\OneDrive\YASH\Sound_Classifcation\heart_sound

09-10-2023  12:22    <DIR>          .
09-10-2023  10:40    <DIR>          ..
06-10-2023  16:43           383,694 classification.ipynb
06-10-2023  12:21                 0 file_names.ipynb
09-10-2023  14:28           187,076 final.ipynb
06-10-2023  12:58            65,009 Meta_Data.csv
09-10-2023  12:22    <DIR>          saved_models
09-10-2023  11:32         2,747,726 test.csv
06-10-2023  13:02    <DIR>          train
06-10-2023  13:04            10,052 unhealthy_class.csv
05-10-2023  13:44    <DIR>          val
               6 File(s)      3,393,557 bytes
               5 Dir(s)  55,824,916,480 bytes free


In [180]:
path = "train/Combined/"
metadata = pd.read_csv("train/Meta_Data.csv")
metadata.tail()

,Filename,class
3233,f0090.wav,unhealthy
3234,f0093.wav,unhealthy
3235,f0096.wav,unhealthy
3236,f0099.wav,unhealthy
3237,f0103.wav,unhealthy


In [181]:
def features_extractor(file):
    audio , sample_rate = librosa.load(file)
    mfccs_features = librosa.feature.mfcc(y=audio , sr= sample_rate , n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T , axis=0)

    return mfccs_scaled_features

In [182]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 44CF-1D40

 Directory of c:\Users\admin\OneDrive\YASH\Sound_Classifcation\heart_sound

09-10-2023  12:22    <DIR>          .
09-10-2023  10:40    <DIR>          ..
06-10-2023  16:43           383,694 classification.ipynb
06-10-2023  12:21                 0 file_names.ipynb
09-10-2023  14:28           187,076 final.ipynb
06-10-2023  12:58            65,009 Meta_Data.csv
09-10-2023  12:22    <DIR>          saved_models
09-10-2023  11:32         2,747,726 test.csv
06-10-2023  13:02    <DIR>          train
06-10-2023  13:04            10,052 unhealthy_class.csv
05-10-2023  13:44    <DIR>          val
               6 File(s)      3,393,557 bytes
               5 Dir(s)  55,824,920,576 bytes free


In [183]:

u = "train/Combined/a0001.wav"

features_extractor(u)


array([-5.6953815e+02,  1.2421279e+02,  8.0652779e+01,  3.1137093e+01,
       -3.3652389e+00, -1.2217492e+01,  5.7084346e-01,  2.0703438e+01,
        3.3236252e+01,  3.1199471e+01,  1.8156874e+01,  3.9651332e+00,
       -2.6544418e+00,  4.4622338e-01,  8.5987177e+00,  1.4636802e+01,
        1.4204312e+01,  8.3054342e+00,  1.7394619e+00, -9.1304368e-01,
        1.4122460e+00,  5.9429750e+00,  8.5860453e+00,  7.1159449e+00,
        2.6347210e+00, -1.5713415e+00, -2.6580801e+00, -2.4827144e-01,
        3.5639341e+00,  6.0066700e+00,  5.6954813e+00,  3.4127977e+00,
        1.1820983e+00,  5.4876953e-01,  1.4778247e+00,  2.6449530e+00,
        2.7255828e+00,  1.5351540e+00,  9.7919136e-02, -3.3093706e-01],
      dtype=float32)

In [184]:
def features_extractor(file):
    path_1="train/Combined/"
    file_name = os.path.join(path_1 , file)
    audio , sample_rate = librosa.load(file_name)
    mfccs_features = librosa.feature.mfcc(y=audio , sr= sample_rate , n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T , axis=0)

    return mfccs_scaled_features

In [185]:
extracted_features = []
for x in os.listdir("train/Combined/"):
    if x.endswith(".wav"):
        data = features_extractor(x)
        extracted_features.append([data])

        

In [186]:
extracted_features_df = pd.DataFrame(extracted_features , columns=["Features"])

In [187]:
extracted_features_df["class"] = metadata["class"]

In [188]:
new_final = extracted_features_df.dropna()

In [189]:
new_final.head()

,Features,class
0,"[-569.53815, 124.21279, 80.65278, 31.137093, -...",healthy
1,"[-620.901, 143.65808, 98.68219, 46.961876, 9.1...",healthy
2,"[-593.1994, 84.55158, 59.88719, 32.74235, 15.1...",healthy
3,"[-558.272, 92.828896, 70.916534, 44.695515, 23...",healthy
4,"[-486.89496, 68.20269, 56.752182, 42.53562, 30...",healthy


In [190]:
 X = np.array(new_final['Features'].tolist())
 y = np.array(new_final['class'].tolist())

In [191]:
X.shape

(3238, 40)

In [192]:
y

array(['healthy', 'healthy', 'healthy', ..., 'unhealthy', 'unhealthy',
       'unhealthy'], dtype='<U9')

In [193]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [194]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train  , y_test = train_test_split(X , y  , test_size=0.2 , random_state=42)

In [195]:
X_train.shape

(2590, 40)

In [196]:
y_test.shape

(648, 2)

In [197]:
import tensorflow as tf

In [198]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Activation , Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [199]:
num_labels = y.shape[1]

In [200]:
y.shape[1]



2

In [201]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [202]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               4100      
                                                                 
 activation_24 (Activation)  (None, 100)               0         
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_25 (Dense)            (None, 200)               20200     
                                                                 
 activation_25 (Activation)  (None, 200)               0         
                                                                 
 dropout_17 (Dropout)        (None, 200)               0         
                                                                 
 dense_26 (Dense)            (None, 100)              

In [203]:
model.compile(loss="binary_crossentropy" , metrics=['accuracy'] , optimizer='adam')

In [207]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

# checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
#                                verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
81/81 [==============================] - 0s 4ms/step - loss: 0.4905 - accuracy: 0.8019 - val_loss: 0.4451 - val_accuracy: 0.8117
Epoch 2/100
81/81 [==============================] - 0s 4ms/step - loss: 0.4846 - accuracy: 0.8015 - val_loss: 0.4340 - val_accuracy: 0.8256
Epoch 3/100
81/81 [==============================] - 0s 4ms/step - loss: 0.4731 - accuracy: 0.8077 - val_loss: 0.4277 - val_accuracy: 0.8287
Epoch 4/100
81/81 [==============================] - 0s 3ms/step - loss: 0.4746 - accuracy: 0.8081 - val_loss: 0.4325 - val_accuracy: 0.8302
Epoch 5/100
81/81 [==============================] - 0s 4ms/step - loss: 0.4825 - accuracy: 0.8035 - val_loss: 0.4258 - val_accuracy: 0.8272
Epoch 6/100
81/81 [==============================] - 0s 4ms/step - loss: 0.4627 - accuracy: 0.8081 - val_loss: 0.4338 - val_accuracy: 0.8210
Epoch 7/100
81/81 [==============================] - 0s 4ms/step - loss: 0.4593 - accuracy: 0.8135 - val_loss: 0.4281 - val_accuracy: 0.8302
Epoch 8/100
8

In [ ]:
# model.fit(X_train , y_train , batch_size=32 , epochs=100 , validation_data=(X_test , y_test))

In [208]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8302469253540039


In [209]:
model.predict(X_test)

21/21 [==============================] - 0s 4ms/step


array([[1.00000000e+00, 1.46737076e-08],
       [9.15034294e-01, 8.49657133e-02],
       [8.90664697e-01, 1.09335326e-01],
       ...,
       [8.90664697e-01, 1.09335326e-01],
       [9.11643386e-01, 8.83565545e-02],
       [8.90664697e-01, 1.09335326e-01]], dtype=float32)

In [210]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

21/21 [==============================] - 0s 3ms/step


In [211]:
filename="train/Combined/a0002.wav"
audio, sample_rate = librosa.load(filename) 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

In [220]:
filename="train/healthy/a0085.wav"
audio, sample_rate = librosa.load(filename) 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

# print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
# print(mfccs_scaled_features)
# print(mfccs_scaled_features.shape)
predict_x=model.predict(mfccs_scaled_features) 
classes_x=np.argmax(predict_x,axis=1)
# predicted_label=model.predict_classes(mfccs_scaled_features)
print(classes_x)
prediction_class = labelencoder.inverse_transform(classes_x) 
prediction_class

1/1 [==============================] - 0s 23ms/step
[0]


array(['healthy'], dtype='<U9')

In [221]:
predict_x

array([[0.8969395 , 0.10306053]], dtype=float32)